In [2]:
import sqlite3
import pandas as pd
import numpy as np
from sqlite3 import Error

In [6]:
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [3]:
def select_all_courses(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM courses")

    rows = cur.fetchall()

    for row in rows:
        print(row)

In [5]:
# essentially a SELECT * FROM courses;
#select_all_courses(create_connection("uts.db"))

In [4]:
def select_course(conn, value):
    """
    Query tasks by priority
    :param conn: the Connection object
    :param priority:
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM courses WHERE course_id LIKE ?", ('%'+value+'%',))
    #cur.execute(f"""SELECT * FROM courses WHERE name='{value}'"""") #sample fstring

    rows = cur.fetchall()

    for row in rows:
        #print(row) #print all rows from database
        print(f"UTS offers {row[1]} at {row[6]}.")

In [5]:
def check_type(x):
    if 'cbk' in x:
        return "Directory"
    elif 'smj' in x:
        return "Directory"
    elif 'maj' in x:
        return "Directory"
    elif 'stm' in x:
        return "Directory"
    elif 'c0' in x:
        return "Course"
    elif 'c1' in x:
        return "Course"
    else:
        return "Subject"

In [6]:
def get_url(x):
    head_url = 'https://handbook.uts.edu.au/'
    if check_type(x) == 'Directory':
        return head_url + 'directory/' + x + '.html'
    elif check_type(x) == 'Course':
        return head_url + 'courses/' + x + '.html'
    elif check_type(x) == 'Subject':
        return head_url + 'subjects/' + x + '.html'
    else:
        return 'No URL found'

In [7]:
def union(conn, value):
    """
    Query tasks by priority
    :param conn: the Connection object
    :param priority:
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT id, name FROM courses WHERE id LIKE ? UNION SELECT id, name FROM sub_structures WHERE id LIKE ?", ('%'+value+'%','%'+value+'%',))
    #cur.execute(f"""SELECT * FROM courses WHERE name='{value}'"""") #sample fstring

    rows = cur.fetchall()

    for row in rows:
        #print(row) #print all rows from database
        print(f"{row[0]} {row[1]}.")
        print(get_url(row[0]))

            


In [36]:
def relations(conn, value):
    cur = conn.cursor()
    cur.execute("SELECT * FROM sub_structures WHERE id IN (SELECT struc_id FROM relations WHERE course_id LIKE ?)", ('%'+value+'%',))
    #cur.execute("SELECT struc_id FROM relations WHERE course_id LIKE ?", ('%'+value+'%',))
    rows = cur.fetchall()
    print(len(rows))
    for row in rows:
        #print(row) #print all rows from database
        print(f"{row[0]} {row[1]}.")

In [40]:
def relations2(conn, value):
    cur = conn.cursor()
    cur.execute("SELECT * FROM courses WHERE id IN (SELECT course_id FROM relations WHERE struc_id LIKE ?)", ('%'+value+'%',))
    #cur.execute("SELECT struc_id FROM relations WHERE course_id LIKE ?", ('%'+value+'%',))
    rows = cur.fetchall()
    print(len(rows))
    for row in rows:
        #print(row) #print all rows from database
        print(f"{row[0]} {row[1]}.")

In [28]:
select_course(create_connection("../uts.db"),"09019")

UTS offers Bachelor of Science (Honours) in Information Technology at Ultimo, Sydney.


In [77]:
union(create_connection("../uts.db"),"437")

maj08437 Accounting.
https://handbook.uts.edu.au/directory/maj08437.html
maj09437 Italy.
https://handbook.uts.edu.au/directory/maj09437.html


In [39]:
relations(create_connection("../uts.db"),"04404")

2
cbk91850 Professional-focused choice.
stm91387 Core Studios stream.


In [44]:
relations2(create_connection("../uts.db"),"31250")

4
c09119 Bachelor of Computing Science (Honours).
c10148 Bachelor of Science in Information Technology.
c10219 Bachelor of Business Bachelor of Science in Information Technology.
c10345 Bachelor of Science in Information Technology Diploma in Information Technology Professional Practice.


In [39]:
second = pd.read_csv("../data/csv/sub_relations.csv")
conn = create_connection("../uts.db")
cur = conn.cursor()
cur.execute("SELECT id FROM sub_structures")

rows = cur.fetchall()

for row in rows:
    #print(row[0])
    i = 0
    for index, r in second.iterrows():
        if r['id'] in row[0]:
            second['id'][i] = row[0]
        i = i + 1

In [42]:
for row in rows:
    #print(row[0])
    i = 0
    for index, r in second.iterrows():
        if r['id2'] in row[0]:
            second['id2'][i] = row[0]
        i = i + 1

In [47]:
second.to_csv("../data/csv/sub_relations.csv", sep=',', index=False)

In [3]:
abcd = pd.read_csv("../data/csv/sub_relations.csv")
abcd

,id,id2
0,cbk90417,cbk90418
1,cbk90417,cbk90483
2,cbk90417,cbk90485
3,cbk90417,cbk90486
4,cbk90418,cbk90490
...,...,...
2965,smj08184,24738
2966,smj08184,24706
2967,smj10087,21832
2968,smj10087,21012


In [4]:

i = 0
for index, r in abcd.iterrows():
    if '-1' in r['id2']:
        abcd['id2'][i] = r['id2'].split('-')[0]
    i = i + 1


In [7]:

conn = create_connection("../uts.db")
cur = conn.cursor()
cur.execute("SELECT id FROM sub_structures")

rows = cur.fetchall()

j = 0
test = []
for row in rows:
    test.append(row[0].split('-')[0])
    j = j + 1

for row in test:
    #print(row[0])
    i = 0
    for index, r in abcd.iterrows():
        if r['id'] in row:
            abcd['id'][i] = row
        if r['id2'] in row:
            abcd['id2'][i] = row
        i = i + 1




In [8]:
abcd

,id,id2
0,cbk90417,cbk90418
1,cbk90417,cbk90483
2,cbk90417,cbk90485
3,cbk90417,cbk90486
4,cbk90418,cbk90490
...,...,...
2965,smj08184,24738
2966,smj08184,24706
2967,smj10087,21832
2968,smj10087,21012


In [9]:
abcd.to_csv("../data/csv/sub_relations.csv", sep=',', index=False)